In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1738/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1738/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
371,103040580200,FRATTA TODINA,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0200,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
364,103040580130,CITTA' DI CASTELLO,CM,M,NaN,7,10,UMBRIA,304,TERNI,058,0130,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
395,103040580440,SAN GIUSTINO,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0440,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0010


Response status code: 200
Processing data for Comune: ASSISI, Provincia: TERNI
STEFANIA PROIETTI: 1616 votes (51,03%)
DONATELLA TESEI: 1465 votes (46,26%)
MARCO RIZZO: 39 votes (1,23%)
MARTINA LEONARDI: 22 votes (0,69%)
MORENO PASQUINELLI: 6 votes (0,19%)
GIUSEPPE PAOLONE: 5 votes (0,16%)
GIUSEPPE TRITTO: 5 votes (0,16%)
FABRIZIO PIGNALBERI: 5 votes (0,16%)
ELIA FRANCESCO FIORINI: 4 votes (0,13%)
Processed ASSISI

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0020


Response status code: 200
Processing data for Comune: BASTIA UMBRA, Provincia: TERNI
STEFANIA PROIETTI: 397 votes (62,23%)
DONATELLA TESEI: 228 votes (35,74%)
GIUSEPPE TRITTO: 6 votes (0,94%)
MARTINA LEONARDI: 4 votes (0,63%)
GIUSEPPE PAOLONE: 1 votes (0,16%)
MARCO RIZZO: 1 votes (0,16%)
FABRIZIO PIGNALBERI: 1 votes (0,16%)
ELIA FRANCESCO FIORINI: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
Processed BASTIA UMBRA

Processing record 3
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0030


Response status code: 200
Processing data for Comune: BETTONA, Provincia: TERNI
DONATELLA TESEI: 190 votes (52,63%)
STEFANIA PROIETTI: 163 votes (45,15%)
MARCO RIZZO: 4 votes (1,11%)
MARTINA LEONARDI: 2 votes (0,55%)
ELIA FRANCESCO FIORINI: 2 votes (0,55%)
GIUSEPPE PAOLONE: 0 votes (0%)
GIUSEPPE TRITTO: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed BETTONA

Processing record 4
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0040


Response status code: 200
Processing data for Comune: BEVAGNA, Provincia: TERNI
DONATELLA TESEI: 110 votes (54,73%)
STEFANIA PROIETTI: 89 votes (44,28%)
GIUSEPPE TRITTO: 2 votes (1,00%)
GIUSEPPE PAOLONE: 0 votes (0%)
MARCO RIZZO: 0 votes (0%)
MARTINA LEONARDI: 0 votes (0%)
ELIA FRANCESCO FIORINI: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed BEVAGNA

Processing record 5
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/058/CM/0050


...

Response status code: 200
Processing data for Comune: PORANO, Provincia: TERNI
GIUSEPPE PAOLONE: 0 votes (0%)
MARCO RIZZO: 0 votes (0%)
MARTINA LEONARDI: 0 votes (0%)
ELIA FRANCESCO FIORINI: 0 votes (0%)
GIUSEPPE TRITTO: 0 votes (0%)
DONATELLA TESEI: 0 votes (0%)
STEFANIA PROIETTI: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed PORANO

Processing record 89
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0290


Response status code: 200
Processing data for Comune: SAN GEMINI, Provincia: TERNI
GIUSEPPE PAOLONE: 0 votes (0%)
MARCO RIZZO: 0 votes (0%)
MARTINA LEONARDI: 0 votes (0%)
ELIA FRANCESCO FIORINI: 0 votes (0%)
GIUSEPPE TRITTO: 0 votes (0%)
DONATELLA TESEI: 0 votes (0%)
STEFANIA PROIETTI: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed SAN GEMINI

Processing record 90
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0300


Response status code: 200
Processing data for Comune: SAN VENANZO, Provincia: TERNI
STEFANIA PROIETTI: 409 votes (52,77%)
DONATELLA TESEI: 349 votes (45,03%)
MARCO RIZZO: 11 votes (1,42%)
ELIA FRANCESCO FIORINI: 3 votes (0,39%)
MARTINA LEONARDI: 2 votes (0,26%)
GIUSEPPE TRITTO: 1 votes (0,13%)
GIUSEPPE PAOLONE: 0 votes (0%)
MORENO PASQUINELLI: 0 votes (0%)
FABRIZIO PIGNALBERI: 0 votes (0%)
Processed SAN VENANZO

Processing record 91
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0310


Response status code: 200
Processing data for Comune: STRONCONE, Provincia: TERNI
DONATELLA TESEI: 919 votes (55,53%)
STEFANIA PROIETTI: 679 votes (41,03%)
MARTINA LEONARDI: 21 votes (1,27%)
MARCO RIZZO: 19 votes (1,15%)
GIUSEPPE PAOLONE: 8 votes (0,48%)
GIUSEPPE TRITTO: 4 votes (0,24%)
MORENO PASQUINELLI: 3 votes (0,18%)
ELIA FRANCESCO FIORINI: 1 votes (0,06%)
FABRIZIO PIGNALBERI: 1 votes (0,06%)
Processed STRONCONE

Processing record 92
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/10/PR/080/CM/0320


Response status code: 200
Processing data for Comune: TERNI, Provincia: TERNI
STEFANIA PROIETTI: 8431 votes (49,99%)
DONATELLA TESEI: 7878 votes (46,71%)
MARCO RIZZO: 225 votes (1,33%)
MARTINA LEONARDI: 167 votes (0,99%)
GIUSEPPE PAOLONE: 49 votes (0,29%)
MORENO PASQUINELLI: 40 votes (0,24%)
ELIA FRANCESCO FIORINI: 35 votes (0,21%)
GIUSEPPE TRITTO: 28 votes (0,17%)
FABRIZIO PIGNALBERI: 14 votes (0,08%)
Processed TERNI

Creating DataFrame...

Sample of processed data:
                  comune provincia regione  \
0                 ASSISI     TERNI  UMBRIA   
1           BASTIA UMBRA     TERNI  UMBRIA   
2                BETTONA     TERNI  UMBRIA   
3                BEVAGNA     TERNI  UMBRIA   
4  CAMPELLO SUL CLITUNNO     TERNI  UMBRIA   

                                                 url  STEFANIA PROIETTI_total  \
0  https://elezioni.interno.gov.it/risultati/2024...                     1616   
1  https://elezioni.interno.gov.it/risultati/2024...                      397   
2  https

In [6]:
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,MORENO PASQUINELLI_total,MORENO PASQUINELLI_perc,GIUSEPPE PAOLONE_total,GIUSEPPE PAOLONE_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc
0,ASSISI,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,1616,"51,03",1465,"46,26",39,"1,23",...,6,"0,19",5,"0,16",5,"0,16",5,"0,16",4,"0,13"
1,BASTIA UMBRA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,397,"62,23",228,"35,74",1,"0,16",...,0,0,1,"0,16",6,"0,94",1,"0,16",0,0
2,BETTONA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,163,"45,15",190,"52,63",4,"1,11",...,0,0,0,0,0,0,0,0,2,"0,55"
3,BEVAGNA,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,89,"44,28",110,"54,73",0,0,...,0,0,0,0,2,"1,00",0,0,0,0
4,CAMPELLO SUL CLITUNNO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,PORANO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,SAN GEMINI,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,SAN VENANZO,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,409,"52,77",349,"45,03",11,"1,42",...,0,0,0,0,1,"0,13",0,0,3,"0,39"
90,STRONCONE,TERNI,UMBRIA,https://elezioni.interno.gov.it/risultati/2024...,679,"41,03",919,"55,53",19,"1,15",...,3,"0,18",8,"0,48",4,"0,24",1,"0,06",1,"0,06"


In [7]:
df_data['perc_diff'] = round(((df_data['STEFANIA PROIETTI_total'] - df_data['DONATELLA TESEI_total']) / df_data['DONATELLA TESEI_total']) * 100, 2)

In [8]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,STEFANIA PROIETTI_total,STEFANIA PROIETTI_perc,DONATELLA TESEI_total,DONATELLA TESEI_perc,MARCO RIZZO_total,MARCO RIZZO_perc,...,MORENO PASQUINELLI_perc,GIUSEPPE PAOLONE_total,GIUSEPPE PAOLONE_perc,GIUSEPPE TRITTO_total,GIUSEPPE TRITTO_perc,FABRIZIO PIGNALBERI_total,FABRIZIO PIGNALBERI_perc,ELIA FRANCESCO FIORINI_total,ELIA FRANCESCO FIORINI_perc,perc_diff
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,1616,"51,03",1465,"46,26",39,"1,23",...,"0,19",5,"0,16",5,"0,16",5,"0,16",4,"0,13",10.31
1,Bastia Umbra,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,397,"62,23",228,"35,74",1,"0,16",...,0,1,"0,16",6,"0,94",1,"0,16",0,0,74.12
2,Bettona,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,163,"45,15",190,"52,63",4,"1,11",...,0,0,0,0,0,0,0,2,"0,55",-14.21
3,Bevagna,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,89,"44,28",110,"54,73",0,0,...,0,0,0,2,"1,00",0,0,0,0,-19.09
4,Campello Sul Clitunno,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Porano,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
88,San Gemini,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
89,San Venanzo,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,409,"52,77",349,"45,03",11,"1,42",...,0,0,0,1,"0,13",0,0,3,"0,39",17.19
90,Stroncone,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,679,"41,03",919,"55,53",19,"1,15",...,"0,18",8,"0,48",4,"0,24",1,"0,06",1,"0,06",-26.12


In [9]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')